In [2]:
import pandas as pd
import numpy as np

import requests

In [3]:
import folium
from folium.plugins import HeatMap
from folium.plugins import MarkerCluster
from folium.plugins import HeatMapWithTime

import os
import sys
 
def generateBaseMap(default_location=[-9.488560, -18.826032], default_zoom_start=1):
    base_map = folium.Map(location=default_location, control_scale=True, zoom_start=default_zoom_start, tiles = "CartoDB positron")
    return base_map

import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px
import plotly.graph_objects as go

In [4]:
pd.set_option('display.max_columns', 20)
pd.set_option('display.width', 20)
pd.set_option('display.float_format', '{:.2f}'.format)
pd.set_option('max_info_columns', 20)

In [5]:
import chart_studio
import chart_studio.plotly as py

In [6]:
teamcolor_dict = { 'N. America East': 'rgb(179, 226, 205)','N. America West': 'rgb(179, 226, 205)', 'N. America': 'rgb(179, 226, 205)',
                  'Europe': 'rgb(223, 226, 177)', 'Central/S. America': 'rgb(226, 177, 199)',  'Asia Pacific': 'rgb(180, 177, 226)'}
color_dict = { 'Yes': '#e07a5f','No': '#3d405b'}

# Import Data 

## Data Historical

In [7]:
data_hist = pd.read_csv("../datasets/01-WFTDA-historical.csv")# Add the name of the file
data_hist.drop(columns = "Unnamed: 0", axis = 1, inplace=True)
data_hist

,GPA,Weigth,Rank,Name,City_old,Region,G,Win,Los,Year,City,CityCode,Country,Longitude,Latitude
0,484.22,1.99,1,Gotham,"New York City, NY, US",N. America,16,16,0,2013-06-01,New York City,NY,USA,-74.01,40.71
1,404.37,1.98,2,Denver,"Denver, CO, US",N. America,15,12,3,2013-06-01,Denver,CO,USA,-104.98,39.74
2,377.71,1.96,3,Oly,"Olympia, WA, US",N. America,11,10,1,2013-06-01,Olympia,WA,USA,-122.90,47.05
3,361.71,1.95,4,Bay_Area,"Oakland, CA, US",N. America,14,11,3,2013-06-01,Oakland,CA,USA,-122.27,37.80
4,334.51,1.94,5,Angel_City,"Los Angeles, CA, US",N. America,17,13,4,2013-06-01,Los Angeles,CA,USA,-118.24,34.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7732,36.70,0.50,350,Hurricane,"Corpus Christi, TX, US",N. America,2,0,2,2019-09-01,Corpus Christi,TX,USA,-97.40,27.75
7733,31.05,0.50,351,Miss_Valley,"LaCrosse, WI, US",N. America,4,0,4,2019-09-01,LaCrosse,WI,USA,-91.10,43.90
7734,28.24,0.50,352,Kaiserslautern,"Rhineland-Pfalz, Germany",Europe,2,0,2,2019-09-01,Rhineland-Pfalz,NaN,Germany,7.84,50.22
7735,15.91,0.50,353,Gothenburg,"Gothenburg, Sweden",Europe,2,0,2,2019-09-01,Gothenburg,NaN,Sweden,11.97,57.71


In [8]:
data_hist = data_hist.sort_values(by=["Year", "Rank"]).reset_index(drop=True)

In [9]:
data_hist["GPA"] = data_hist.GPA.str.replace(",", "")
data_hist["GPA"] = data_hist.GPA.astype(float)

In [10]:
data_hist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7737 entries, 0 to 7736
Data columns (total 15 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   GPA        7737 non-null   float64
 1   Weigth     7737 non-null   float64
 2   Rank       7737 non-null   int64  
 3   Name       7737 non-null   object 
 4   City_old   7737 non-null   object 
 5   Region     7737 non-null   object 
 6   G          7737 non-null   int64  
 7   Win        7737 non-null   int64  
 8   Los        7737 non-null   int64  
 9   Year       7737 non-null   object 
 10  City       7737 non-null   object 
 11  CityCode   6190 non-null   object 
 12  Country    7737 non-null   object 
 13  Longitude  7737 non-null   float64
 14  Latitude   7737 non-null   float64
dtypes: float64(4), int64(4), object(7)
memory usage: 906.8+ KB


## Time Line

In [11]:
dfcount = pd.read_csv("../datasets/01-timeseries.csv")# Add the name of the file
dfcount.drop(columns = "Unnamed: 0", axis = 1, inplace=True)
dfcount.columns  = ['Year', 'Region', 'count']

In [12]:
dfcount = dfcount.sort_values(by=["Year", "Region"]).reset_index(drop=True)
dfcount

,Year,Region,count
0,2013-06-01,Asia Pacific,1
1,2013-06-01,Central/S. America,0
2,2013-06-01,Europe,12
3,2013-06-01,N. America East,88
4,2013-06-01,N. America West,65
...,...,...,...
130,2019-12-01,Asia Pacific,17
131,2019-12-01,Central/S. America,2
132,2019-12-01,Europe,71
133,2019-12-01,N. America East,159


## Arenas

In [13]:
arena = pd.read_csv("../datasets/01-HomeArenas.csv")# Add the name of the file
arena.drop(columns = "Unnamed: 0", axis = 1, inplace=True)
arena

,Lat_Long,Cantidad,Latitude,Longitude,Region
0,"40.1582, -75.0152",53,40.16,-75.02,N. America
1,"44.0505, -123.0951",48,44.05,-123.10,N. America
2,"48.8567, 2.3515",46,48.86,2.35,Europe
3,"45.4972, -73.6104",44,45.50,-73.61,N. America
4,"60.1674, 24.9426",42,60.17,24.94,Europe
...,...,...,...,...,...
327,"41.5538, -73.0438",1,41.55,-73.04,N. America
328,"-41.2888, 174.7772",1,-41.29,174.78,Asia Pacific
329,"42.5083, -89.0318",1,42.51,-89.03,N. America
330,"40.7127, -74.0060",1,40.71,-74.01,N. America


## Games

In [14]:
game = pd.read_csv("../datasets/01-Games_.csv")# Add the name of the file
game.drop(columns = "Unnamed: 0", axis = 1, inplace=True)
game

,Date,H_Team,V_Team,Tournament,WFTDA_H,WFTDA_V,City_Home,Region_Home,Longitude_Home,Latitude_Home,...,Latitude_Visi,Ranked_V,Dates,Class,YES,Country_Home,Maps,month,week,day_name
0,2019-02-16,Fernie_Roller_Derby,Gorge,0,Fernie,Gorge,"Fernie, BC, Canada",N. America,-115.06,49.50,...,45.71,1,NaN,NaN,NaN,Canada,"49.5042, -115.0629",2,7,5
1,2019-02-16,Bellingham,Eves_Destruct,0,Bellingham,Eves_Destruct,"Bellingham, WA, US",N. America,-122.48,48.75,...,48.43,1,NaN,NaN,NaN,USA,"48.7544, -122.4788",2,7,5
2,2019-02-16,Fernie_Roller_Derby,Bellingham,0,Fernie,Bellingham,"Fernie, BC, Canada",N. America,-115.06,49.50,...,48.75,1,NaN,NaN,NaN,Canada,"49.5042, -115.0629",2,7,5
3,2019-02-16,Gorge,Eves_Destruct,0,Gorge,Eves_Destruct,"Hood River, OR, US",N. America,-121.51,45.71,...,48.43,1,NaN,NaN,NaN,USA,"45.7088, -121.5124",2,7,5
4,2019-02-16,Roc_City,Orangeville,0,Roc_City,Orangeville,"Rochester, NY, US",N. America,-77.62,43.16,...,43.92,0,NaN,NaN,NaN,USA,"43.1573, -77.6152",2,7,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1795,2019-11-23,Namur,Leeds,Skate_Around_Tour,Namur,Leeds,"Namur, Belgium",Europe,4.87,50.47,...,53.80,1,11/23/19 - 11/24/19,Invitational,Skate_Around_Tour,Belgium,"50.4665, 4.8662",11,47,5
1796,2019-11-23,Namur,Royal Swedish,Skate_Around_Tour,Namur,Royal_Swedish,"Namur, Belgium",Europe,4.87,50.47,...,59.33,1,11/23/19 - 11/24/19,Invitational,Skate_Around_Tour,Belgium,"50.4665, 4.8662",11,47,5
1797,2019-11-23,Namur,Nantes,Skate_Around_Tour,Namur,Nantes,"Namur, Belgium",Europe,4.87,50.47,...,47.22,1,11/23/19 - 11/24/19,Invitational,Skate_Around_Tour,Belgium,"50.4665, 4.8662",11,47,5
1798,2019-11-23,Namur,Namur,Skate_Around_Tour,Namur,Namur,"Namur, Belgium",Europe,4.87,50.47,...,50.47,1,11/23/19 - 11/24/19,Invitational,Skate_Around_Tour,Belgium,"50.4665, 4.8662",11,47,5


In [15]:
game = game.sort_values(by=["Date", "Tournament"]).reset_index(drop=True)

In [16]:
game.drop(columns=['Ranked_H','Ranked_V', "H_Team", "V_Team"], axis=1, inplace=True)

In [17]:
game.loc[(game.City_Home == "South Australia"), "Region_Home"] = 'Asia Pacific'

In [18]:
game = game

In [19]:
game.loc[(game.WFTDA_H == "Rainy_City") & (game.City_Home == "Centralia, WA, US") , "WFTDA_H"] = "Rainy_City_Roller_Dolls"
game.loc[(game.WFTDA_V == "Rainy_City") & (game.City_Visi == "Centralia, WA, US") , "WFTDA_V"] = "Rainy_City_Roller_Dolls"


In [20]:
game = game.loc[game.WFTDA_V != game.WFTDA_H ,:]

# DATAHIST

In [21]:
# I have a few cities repeat a lot more than the most frecuent in City_old column 
#Denver, CO, US (54) vs London (74) / Birmingham (68) / Melbourne (64)

print(data_hist.loc[(data_hist.City == "London"), "City_old"].unique())
print(data_hist.loc[(data_hist.City == "Birmingham"), "City_old"].unique())
print(data_hist.loc[(data_hist.City == "Melbourne"), "City_old"].unique())

['London, England' 'London, ON, Canada']
['Birmingham, England' 'Birmingham, AL, US']
['Melbourne, VIC, Australia' 'Melbourne, FL, US']


In [22]:
data_hist["Fundation"]= data_hist.Name.apply(lambda x: data_hist.loc[(data_hist.Name == x), "Year"].min())

In [23]:
data_hist["Fund_Year"]= data_hist.Fundation.apply(lambda x: x[0:4])

In [24]:
df = data_hist.loc[(data_hist.Year == "2019-12-01"), :]
df.Fundation.nunique()

27

In [26]:
data_hist["count"]= 1

## Map Country Historic (HeatMap Animation)

In [27]:
df_month_list = []
for year in data_hist.Year.sort_values().unique():
    df_month_list.append(data_hist.loc[data_hist.Year == year, ['Latitude', 'Longitude', 'count']].groupby(['Latitude', 'Longitude']).sum().reset_index().values.tolist())
    

mapa_hist = generateBaseMap(default_zoom_start=2)
HeatMapWithTime(df_month_list, radius=10, min_opacity=0.5, max_opacity=0.8, use_local_extrema=True).add_to(mapa_hist)
mapa_hist

In [37]:
mapa_hist.save("../Fig/HIST_TEAM.html")

## Map Year Fundation (MarkerCircle 12/2019)

In [29]:
mappin = {'2013-06-01' : 90, '2013-09-01' : 90 ,'2013-12-01' : 90,
         '2014-03-01' : 75, '2014-06-01' : 75, '2014-09-01' : 75 , '2014-12-01' : 75 ,
         '2015-03-01' : 60, '2015-06-01' : 60, '2015-09-01' : 60, '2015-12-01' : 60,
         '2016-03-01' : 45, '2016-06-01' : 45, '2016-09-01' : 45, '2016-12-01' : 45, 
         '2017-03-01' : 30, '2017-06-01' : 30, '2017-09-01' : 30, '2017-12-01' : 30,
         '2018-03-01' : 15 ,'2018-06-01' : 15, '2018-09-01' : 15,  '2018-12-01' : 15,
         '2019-03-01' : 1, '2019-06-01' : 1, '2019-09-01' : 1, '2019-12-01' : 1   
         }
df['Size'] = df['Fundation'].map(mappin) 

d:\anaconda\envs\dataanalytic\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [30]:
# Create base map
mapa1 = folium.Map(location = [-9.488560, -18.826032],
                 zoom_start = 1.5, 
                 tiles = "CartoDB positron") # Sets Tile Theme to (Dark Theme)

mc = MarkerCluster()
for index, row in df.iterrows():
    mc.add_child(folium.Marker(location=[str(row['Latitude']),     str(row['Longitude'])], tooltip= str(row['Name'] + " in the ranking since  "+ row['Fundation']),
                               icon=folium.Icon(color="darkpurple", icon="star",      clustered_marker=True)))
    folium.Circle(location=[str(row['Latitude']), str(row['Longitude'])], color='pink', fill=False,  radius=int(row['Size'])*900).add_to(mapa1)
mapa1.add_child(mc)
#mapa1.save('medium_map.html') # Saves Map to an HTML File

##  Rank

In [51]:
rank = data_hist.loc[(data_hist.Rank < 51), :].sort_values(by=(["Year", "Weigth"]), ascending=True).reset_index(drop= True)
rank

,GPA,Weigth,Rank,Name,City_old,Region,G,Win,Los,Year,City,CityCode,Country,Longitude,Latitude,Fundation,Fund_Year,count
0,171.80,1.40,50,Carolina,"Raleigh, NC, US",N. America,13,3,10,2013-06-01,Raleigh,NC,USA,-78.64,35.78,2013-06-01,2013,1
1,177.68,1.41,49,DC,"Washington, DC, US",N. America,19,7,12,2013-06-01,Washington,DC,USA,-77.04,38.89,2013-06-01,2013,1
2,179.15,1.42,48,Blue_Ridge,"Asheville, NC, US",N. America,14,6,8,2013-06-01,Asheville,NC,USA,-82.55,35.60,2013-06-01,2013,1
3,181.00,1.43,47,Mid_Iowa,"Des Moines, IA, US",N. America,2,2,0,2013-06-01,Des Moines,IA,USA,-93.60,41.59,2013-06-01,2013,1
4,181.51,1.45,46,Kalamazoo,"Kalamazoo, MI, US",N. America,18,11,7,2013-06-01,Kalamazoo,MI,USA,-85.59,42.29,2013-06-01,2013,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1345,1154.97,8.01,5,Gotham,"New York City, NY, US",N. America,12,9,3,2019-12-01,New York City,NY,USA,-74.01,40.71,2013-06-01,2013,1
1346,1158.81,8.04,4,Arch_Rival,"St. Louis, MO, US",N. America,10,6,4,2019-12-01,St. Louis,MO,USA,-90.24,38.65,2013-06-01,2013,1
1347,1171.56,8.13,3,Denver,"Denver, CO, US",N. America,13,8,5,2019-12-01,Denver,CO,USA,-104.98,39.74,2013-06-01,2013,1
1348,1228.65,8.52,2,Victoria,"Melbourne, VIC, Australia",Asia Pacific,10,6,4,2019-12-01,Melbourne,NaN,Australia,144.96,-37.81,2013-06-01,2013,1


In [54]:
data_hist = data_hist.sort_values(by=(["Year", "Weigth"]), ascending=True).reset_index(drop= True)

# DF COUNT

# GAMES 2019

In [66]:
game.loc[game.WFTDA_H == ('CoMo'), "City_Home"] = "Jefferson City, MO, US"
game.loc[game.WFTDA_V == ('CoMo'), "City_Visi"] = "Jefferson City, MO, US"

In [67]:
game.loc[(game.WFTDA_V == ("Convict_City")), "City_Visi"] = "Hobart, TAS, Australia"
game.loc[(game.WFTDA_H == ("Convict_City")), "City_Home"] = "Hobart, TAS, Australia"

In [68]:
mappin = {
    'Birmingham, England' : 'Birmingham, England, UK',
    'E Sussex, England' : 'E Sussex, England, UK',
    'Glasgow, UK' : 'Glasgow, Scotland, UK',
    'Glasgow, Scotland' : 'Glasgow, Scotland, UK',
    'Hull, UK' : 'Hull, England, UK', 
    'Herne Bay, Kent, UK' :   'Herne Bay, England, UK',
    'Leeds, England' : 'Leeds, England, UK',
    'Leicestershire, England' : 'Leicestershire, England, UK',
    'London, England' : 'London, England, UK', 
    'Manchester, England' : 'Manchester, England, UK',
    'Manchester, UK' : 'Manchester, England, UK',
    'Middlesbrough, England' : 'Middlesbrough, England, UK',
    'Newcastle, England' : 'Newcastle, England, UK',
    'Nottingham, England' : 'Nottingham, England, UK',
    'Sheffield, England' : 'Sheffield, England, UK',
    'Windsor, UK, United Kingdom' : 'Windsor, England, UK',
    'Rennes, France, France' : 'Rennes, France',
    'Nice, France, France' : 'Nice, France',
    'Saint-Anselme, Quebec, Canada' : 'Saint-Anselme, QC, Canada',
    'Whitby, Ontario, Canada' : 'Whitby, ON, Canada',
    'Perth, Australia' : 'Perth, WA, Australia',
    'Golden Beach, Queensland, Australia' : 'Golden Beach, QLD, Australia',
    'Brisbane, Australia' : 'Brisbane, QLD, Australia',
    'Dundee, Scotland' : 'Dundee, Scotland, UK',
    'Swansea, Wales' : 'Swansea, Wales, UK',
    'Grey Lynn, Auckland, New Zealand' : 'Grey Lynn, New Zealand',
    'Belfast, Northern Ireland' : 'Belfast, Northern Ireland, UK',
    'Edinburgh, Scotland' : 'Edinburgh, Scotland, UK',
    'The Hague, Netherlands, Netherlands' : 'The Hague, Netherlands',
    'Turin, Italy, Italy' : 'Turin, Italy',
    'Oulu, Finland, Finland' : 'Oulu, Finland',
    'Milan, Italy, Italy' : 'Milan, Italy', 
    'Cardiff, Wales' : 'Cardiff, Wales, UK',
    'MD' : 'Hagerstown, MD, US', 
    
}

game.City_Home  = game.City_Home.replace(mappin) 
game.City_Visi  = game.City_Visi.replace(mappin) 

In [69]:
game.City_Home = game.City_Home.str.replace(", USA", ", US")
game.City_Visi = game.City_Visi.str.replace(", USA", ", US")

In [70]:
game[game.City_Visi.str.contains("Jef")]

,Date,Tournament,WFTDA_H,WFTDA_V,City_Home,Region_Home,Longitude_Home,Latitude_Home,City_Visi,Region_Visi,Longitude_Visi,Latitude_Visi,Dates,Class,YES,Country_Home,Maps,month,week,day_name
623,2019-05-04,0,Quad_City,CoMo,"Bettendorf, IA, US",N. America,-90.51,41.53,"Jefferson City, MO, US",N. America,-92.56,38.76,NaN,NaN,NaN,USA,"41.5253, -90.5096",5,18,5
1216,2019-06-22,0,Roughneck,CoMo,"Tulsa, OK, US",N. America,-95.99,36.16,"Jefferson City, MO, US",N. America,-92.56,38.76,NaN,NaN,NaN,USA,"36.1557, -95.9929",6,25,5


In [71]:
game.YES = game.YES.fillna("No")
game.loc[(game.YES != "No"), "YES"] = "Yes"

In [72]:
games = game.loc[ (game.Tournament == "0" ), :]
tournaments = game.loc[ (game.Tournament != "0"), :]

In [73]:
game.Date = pd.to_datetime(game.Date, format='%Y-%m-%d')

## Games by Dates

## Tournaments

## Map Arena Month (HeatMap Animation)

In [167]:
game["count"]=1
df_month_list = []
for month in game.month.sort_values().unique():
    df_month_list.append(game.loc[game.month == month, ['Latitude_Home', 'Longitude_Home', 'count']].groupby(['Latitude_Home', 'Longitude_Home']).sum().reset_index().values.tolist())
    


gamesbymonth = generateBaseMap(default_zoom_start=2)
HeatMapWithTime(df_month_list, radius=10, min_opacity=0.5, max_opacity=0.8, use_local_extrema=True).add_to(gamesbymonth)
games

d:\anaconda\envs\dataanalytic\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [169]:
gamesbymonth.save("../Fig/Map_gamesbymonth.html")

## Map Arenas (HeatMap)

In [157]:
df_copy = arena.copy()

Tournamens = generateBaseMap()
HeatMap(data=df_copy[['Latitude', 'Longitude', 'Cantidad']].groupby(['Latitude', 'Longitude']).sum().reset_index().values.tolist(), radius=8, max_zoom=13).add_to(Tournamens)
Tournamens

In [168]:
Tournamens.save("../Fig/Map_Tournaments.html")

In [166]:
locations = df_copy[['Latitude', 'Longitude']]
locationlist = locations.values.tolist()
len(locationlist)


332